In [56]:
import pandas as pd
import numpy as np
import os
import random
import matplotlib.pyplot as plt

In [57]:
# Get paths to working directory and files folder
dir_path = os.path.dirname(os.path.abspath(os.curdir))
dir_path = os.path.join(dir_path, 'Mutational_Signatures')
print(dir_path)

c:\Files\github\Mutational_Signatures


In [58]:
# Read in the tables
cosmic_signatures_path = os.path.join(dir_path, 'cosmic_signatures')
arr = os.listdir(cosmic_signatures_path)
data_frames = []
data_frames_names = []
for file in arr:
    data_frames.append(pd.read_csv(os.path.join(cosmic_signatures_path, file)))
    data_frames_names.append(file.removesuffix('.csv'))

print(data_frames[1].to_string)
print(data_frames_names)

<bound method DataFrame.to_string of    Mutation  SBS4_GRCh37  SBS4_GRCh38  SBS4_mm9  SBS4_mm10  SBS4_rn6
0   A[C>A]A     0.042196     0.042451  0.047600   0.047599  0.047990
1   A[C>A]C     0.033297     0.032990  0.033653   0.033655  0.034018
2   A[C>A]G     0.015599     0.016116  0.014097   0.014094  0.016775
3   A[C>A]T     0.029498     0.029663  0.029589   0.029590  0.029143
4   A[C>G]A     0.006889     0.006931  0.007772   0.007771  0.007835
..      ...          ...          ...       ...        ...       ...
91  T[T>C]T     0.000636     0.000639  0.000544   0.000544  0.000536
92  T[T>G]A     0.000377     0.000372  0.000337   0.000337  0.000335
93  T[T>G]C     0.000174     0.000177  0.000178   0.000178  0.000179
94  T[T>G]G     0.002320     0.002339  0.002272   0.002272  0.002242
95  T[T>G]T     0.000568     0.000570  0.000485   0.000485  0.000479

[96 rows x 6 columns]>
['SBS18', 'SBS4', 'SBS6', 'SBS7a', 'SBS9']


In [59]:
df_grch37 = pd.DataFrame()
for i in range(len(data_frames_names)):
    column = data_frames[i][data_frames_names[i] + '_GRCh37']
    df_grch37 = pd.concat([df_grch37, column], axis = 1)
df_grch37

,SBS18_GRCh37,SBS4_GRCh37,SBS6_GRCh37,SBS7a_GRCh37,SBS9_GRCh37
0,0.051534,0.042196,0.000425,6.704351e-05,0.000558
1,0.015810,0.033297,0.000524,1.791162e-04,0.004090
2,0.002432,0.015599,0.000052,7.124623e-05,0.000426
3,0.021414,0.029498,0.000180,2.481610e-04,0.003050
4,0.001731,0.006889,0.000471,6.494214e-05,0.004840
...,...,...,...,...,...
91,0.003372,0.000636,0.001741,8.125273e-04,0.038397
92,0.000696,0.000377,0.000105,1.280831e-04,0.065695
93,0.002101,0.000174,0.000287,1.160753e-04,0.008619
94,0.001451,0.002320,0.000324,2.231448e-16,0.010899


In [60]:
n_samples = 10
df_sparse = pd.DataFrame()
for i in range(n_samples):
    # TODO: find what distribution of signatures to use (Article uses 5 out of 10 for each sample)
    # Right now, use 0.4 percent chance of signatures being present, with the strength of it being between 0.5 and 2

    # Get a distribution of the counts
    distribution = [random.random()*1.5+0.5 if random.uniform(0, 1) > 0.6 else 0 for x in range(0, df_grch37.shape[1])]
    total = sum(distribution)
    while(sum(distribution)==0):
        distribution = [random.random()*1.5+0.5 if random.uniform(0, 1) > 0.6 else 0 for x in range(0, df_grch37.shape[1])]
        total = sum(distribution)

    # normalize
    total = sum(distribution)
    distribution = [x/total for x in distribution]
    df_sparse[i] = distribution

    # TODO: add Poisson noise
    np.random.poisson(0.2)



df_sparse = df_sparse.set_index(df_grch37.columns)
df_sparse

,0,1,2,3,4,5,6,7,8,9
SBS18_GRCh37,0.000000,0.000000,0.000000,0.331900,0.614643,0.000000,0.416035,0.000000,0.0,0.00000
SBS4_GRCh37,0.000000,0.000000,0.365398,0.000000,0.000000,0.000000,0.148800,0.000000,0.0,0.00000
SBS6_GRCh37,0.378908,0.532964,0.000000,0.335169,0.000000,0.483108,0.435166,0.000000,0.0,0.45237
SBS7a_GRCh37,0.000000,0.467036,0.000000,0.000000,0.385357,0.516892,0.000000,0.615991,0.0,0.54763
SBS9_GRCh37,0.621092,0.000000,0.634602,0.332931,0.000000,0.000000,0.000000,0.384009,1.0,0.00000


In [61]:
average_noise = 10

simulated_data = df_grch37.dot(df_sparse)
for i in range(simulated_data.shape[1]):
    distribution = simulated_data[i]
    # Get the number of counts between 1001 and 50119 in logscale (50119 for easier numbers in formula)
    n_counts = 10 ** (random.uniform(3, 4.7))
    counts = [int(x*n_counts) for x in distribution]

    # Add Poisson noise
    noisy_counts = [x+np.random.poisson(average_noise) for x in counts]
    simulated_data[i] = noisy_counts

simulated_data

,0,1,2,3,4,5,6,7,8,9
0,12,18,125,39,92,10,208,17,13,15
1,40,15,117,23,34,3,97,55,30,7
2,12,13,53,8,14,10,36,15,13,11
3,29,22,98,23,44,9,106,51,18,9
4,44,16,43,12,12,6,22,62,27,12
...,...,...,...,...,...,...,...,...,...,...
91,267,43,190,29,10,15,27,433,151,15
92,442,14,316,42,10,9,14,715,250,8
93,65,15,53,12,17,12,20,101,36,12
94,78,14,63,19,10,11,14,125,54,13


only simulate 30 samples (since real-life databases are also small (not in our case though??))

1. Choose 5 random signatures per sample to get a representation in combined probability (not really anymore since not sum to 100%
2. randomly select between 1000 to 50000 mutations in log scale per sample
3. Calculate the counts of the samples
4. Add Poisson noise